### 165 calculation target loyalty feature
targetとなっているloyaltyを計算している期間に絞ったfeatureを作成する

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
# df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
# df_hist = reduce_mem_usage(df_hist)
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
df_new = reduce_mem_usage(df_new)

100%|██████████| 3/3 [00:01<00:00,  2.75it/s]


Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [3]:
# term
col_month = 'month_lag'

new_cat1 = df_new[df_new.category_1==1]
new_cat0 = df_new[df_new.category_1==0]

### Null Count

In [4]:
last_type = 'new'
null_cols = ['category_2', 'installments', 'merchant_id']

df_list = [new_cat1, new_cat0]

# category_2
col = 'category_2'
for df in df_list:
    df[col].fillna('NA', inplace=True)
fname = 'new_cat0'
new_cat0[f'{col}_null_cnt'] = (new_cat0[col]=='NA')

# installments
col = 'installments'
fname = 'new_cat0'
new_cat0[f'{col}_isnull'] = (new_cat0[col].isnull())
fname = 'new_cat1'
new_cat1[f'{col}_isnull'] = (new_cat1[col].isnull())

# merchant_id
col = 'merchant_id'
fname = 'new_cat0'
new_cat0[f'{col}_isnull'] = (new_cat0[col].isnull())
fname = 'new_cat1'
new_cat1[f'{col}_isnull'] = (new_cat1[col].isnull())

In [5]:
last_type = 'new'
df_list = [new_cat1, new_cat0]
fname_list = [f'{last_type}_cat1', f'{last_type}_cat0']
train_list = []
test_list = []

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


def feat_agg(df, fname):

    null_cols = [col for col in df.columns if col.count('isnull')]
    
    aggs = {}
    
    # category_1==1のときcategory_2 = Null
    if fname.count('cat0'):
        cat_list = ['category_2']
        df = get_dummies(df=df, cat_list=cat_list)
        category_2_list = [col for col in df.columns if col.count('category_2')]
        for cat in category_2_list:
            aggs[cat] = ['mean', 'sum']
            
    # isnull feature
    for col in null_cols:
        aggs[col] = ['mean', 'sum']
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    df['unix_date'] =  df['purchase_date'].map(lambda x: int(x.timestamp()) )
    
    aggs['month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum','max', 'min','mean', 'std']
    aggs['unix_date'] = ['max','min']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    aggs['card_id'] = ['size']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
    
    df_agg[f'{fname}_term'] = (df_agg[f'{fname}_unix_date_max'] - df_agg[f'{fname}_unix_date_min']).astype('int64')
    df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
    df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
    df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / df_agg[f'{fname}_installments_sum']
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / df_agg[f'{fname}_installments_mean']
    
    feat_train = df_train.merge(df_agg, on=key, how='left')
    feat_test = df_test.merge(df_agg, on=key, how='left')
    del df_agg
    gc.collect()
    
    print(fname)
    display(feat_train.head())
    
    train_list.append(feat_train)
    test_list.append(feat_test)
    
for df, fname in zip(df_list, fname_list):
    feat_agg(df, fname)

new_cat1


,card_id,first_active_month,feature_1,feature_2,feature_3,target,new_cat1_installments_isnull_mean,new_cat1_installments_isnull_sum,new_cat1_merchant_id_isnull_mean,new_cat1_merchant_id_isnull_sum,new_cat1_month_lag_mean,new_cat1_month_lag_std,new_cat1_yyyy_week_nunique,new_cat1_purchase_amount_sum,new_cat1_purchase_amount_max,new_cat1_purchase_amount_min,new_cat1_purchase_amount_mean,new_cat1_purchase_amount_over_sum,new_cat1_purchase_amount_over_max,new_cat1_purchase_amount_over_min,new_cat1_purchase_amount_over_mean,new_cat1_installments_sum,new_cat1_installments_max,new_cat1_installments_min,new_cat1_installments_mean,new_cat1_installments_std,new_cat1_unix_date_max,new_cat1_unix_date_min,new_cat1_purchase_date_max,new_cat1_purchase_date_min,new_cat1_month_diff_mean,new_cat1_month_diff_std,new_cat1_card_id_size,new_cat1_merchant_id_nunique,new_cat1_merchant_category_id_nunique,new_cat1_term,new_cat1_freq_per_90days,new_cat1_amount_per_90days,new_cat1_installments_per_90days,new_cat1_amount_per_installments_sum,new_cat1_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,0.0,0.0,False,False,2.0,NaN,1.0,0.083984,0.083984,0.083984,0.083984,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,1.524049e+09,1.524049e+09,2018-04-18 11:00:11,2018-04-18 11:00:11,12.0,NaN,1.0,1.0,1.0,0.0,0.011111,0.000933,0.011108,0.083984,0.083984
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,0.0,0.0,False,False,1.5,0.707107,2.0,0.645508,0.417725,0.227539,0.322754,0.0,0.0,0.0,0.0,3.0,2.0,1.0,1.5,0.707031,1.523043e+09,1.521291e+09,2018-04-06 19:28:07,2018-03-17 12:43:08,34.0,14.142136,2.0,2.0,2.0,1752299.0,0.022222,0.007172,0.033325,0.215210,0.215210


new_cat0


,card_id,first_active_month,feature_1,feature_2,feature_3,target,new_cat0_category_2_null_cnt_mean,new_cat0_category_2_null_cnt_sum,new_cat0_dummie_category_2_1.0_mean,new_cat0_dummie_category_2_1.0_sum,new_cat0_dummie_category_2_2.0_mean,new_cat0_dummie_category_2_2.0_sum,new_cat0_dummie_category_2_3.0_mean,new_cat0_dummie_category_2_3.0_sum,new_cat0_dummie_category_2_4.0_mean,new_cat0_dummie_category_2_4.0_sum,new_cat0_dummie_category_2_5.0_mean,new_cat0_dummie_category_2_5.0_sum,new_cat0_dummie_category_2_NA_mean,new_cat0_dummie_category_2_NA_sum,new_cat0_installments_isnull_mean,new_cat0_installments_isnull_sum,new_cat0_merchant_id_isnull_mean,new_cat0_merchant_id_isnull_sum,new_cat0_month_lag_mean,new_cat0_month_lag_std,new_cat0_yyyy_week_nunique,new_cat0_purchase_amount_sum,new_cat0_purchase_amount_max,new_cat0_purchase_amount_min,new_cat0_purchase_amount_mean,new_cat0_purchase_amount_over_sum,new_cat0_purchase_amount_over_max,new_cat0_purchase_amount_over_min,new_cat0_purchase_amount_over_mean,new_cat0_installments_sum,new_cat0_installments_max,new_cat0_installments_min,new_cat0_installments_mean,new_cat0_installments_std,new_cat0_unix_date_max,new_cat0_unix_date_min,new_cat0_purchase_date_max,new_cat0_purchase_date_min,new_cat0_month_diff_mean,new_cat0_month_diff_std,new_cat0_card_id_size,new_cat0_merchant_id_nunique,new_cat0_merchant_category_id_nunique,new_cat0_term,new_cat0_freq_per_90days,new_cat0_amount_per_90days,new_cat0_installments_per_90days,new_cat0_amount_per_installments_sum,new_cat0_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,1.000000,23.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,False,False,1.478261,0.510754,7.0,3.939453,0.450928,0.022476,0.171265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.525001e+09,1.520259e+09,2018-04-29 11:23:05,2018-03-05 14:04:36,29.739130,16.591941,23.0,23.0,14.0,4742309.0,0.255556,0.043762,0.000000,inf,inf
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,0.0,0.0,1.000000,6.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,False,False,1.500000,0.547723,4.0,0.126953,0.045410,0.007820,0.021164,0.0,0.0,0.0,0.0,6.0,1.0,1.0,1.0,0.0,1.522393e+09,1.517505e+09,2018-03-30 06:48:26,2018-02-01 17:07:54,61.500000,26.402651,6.0,6.0,5.0,4887632.0,0.066667,0.001410,0.066650,0.021164,0.021164
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.0,1.0,0.0,0.0,0.000000,0.0,False,False,2.000000,NaN,1.0,0.046875,0.046875,0.046875,0.046875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.524937e+09,1.524937e+09,2018-04-28 17:43:11,2018-04-28 17:43:11,2.000000,NaN,1.0,1.0,1.0,0.0,0.011111,0.000521,0.000000,inf,inf
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,6.0,0.0,0.0,0.0,0.0,0.166667,1.0,False,False,1.666667,0.516398,5.0,0.490723,0.180176,0.012703,0.081787,0.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,0.0,1.523949e+09,1.520424e+09,2018-04-17 07:15:40,2018-03-07 11:55:06,28.000000,15.270887,6.0,6.0,5.0,3525634.0,0.066667,0.005451,0.055542,0.098145,0.081787
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,0.0,0.0,0.058824,2.0,0.0,0.0,0.205882,7.0,0.735294,25.0,0.0,0.0,0.0,0.0,0.029412,1.0,False,False,1.558824,0.503995,8.0,6.324219,1.198242,0.007820,0.186035,0.0,0.0,0.0,0.0,33.0,1.0,1.0,1.0,0.0,1.524941e+09,1.519992e+09,2018-04-28 18:50:25,2018-03-02 11:55:43,28.911765,17.272096,34.0,34.0,15.0,4949682.0,0.377778,0.070251,0.366699,0.191650,0.186035


In [6]:
# impute

def impute_agg(df):
    for col in df.columns:
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
        
for train, test in zip(train_list, test_list):
    try:
        train.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month'], axis=1, inplace=True)
    except KeyError:
        pass
    
    try:
        train.set_index(key, inplace=True)
        test.set_index(key, inplace=True)
    except KeyError:
        pass
    
    df_train = df_train.join(train)
    df_test = df_test.join(test)

df_train = impute_agg(df_train)
df_test = impute_agg(df_test)

In [7]:
last_type = 'new'

df_train['first_active_month'] =  pd.to_datetime(df_train['first_active_month'])
df_test['first_active_month'] =  pd.to_datetime(df_test['first_active_month'])

# 最終のcat0, newまでの期間
df_train[f'{last_type}_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# 最終のcat1, newまでの期間
df_train[f'{last_type}_cat0_elapsed_time_from_first'] = (df_train[f'{last_type}_cat0_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_elapsed_time_from_first']  = (df_test[f'{last_type}_cat0_purchase_date_max'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_elapsed_time_from_first'] = (df_train[f'{last_type}_cat1_purchase_date_max'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_elapsed_time_from_first']  = (df_test[f'{last_type}_cat1_purchase_date_max'] - df_test[f'first_active_month']).dt.days

# {last_type}の初回購買までの期間
df_train[f'{last_type}_cat0_first_buy'] = (df_train[f'{last_type}_cat0_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat0_first_buy']  = (df_test[f'{last_type}_cat0_purchase_date_min'] - df_test[f'first_active_month']).dt.days
df_train[f'{last_type}_cat1_first_buy'] = (df_train[f'{last_type}_cat1_purchase_date_min'] - df_train[f'first_active_month']).dt.days
df_test[f'{last_type}_cat1_first_buy']  = (df_test[f'{last_type}_cat1_purchase_date_min'] - df_test[f'first_active_month']).dt.days

# 20180501までの期間を各データセットパターンで
df_train[f'{last_type}_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_cat0_purchase_date_max']).dt.days
df_train[f'{last_type}_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_train[f'{last_type}_cat1_purchase_date_max']).dt.days
df_test[f'{last_type}_cat0_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_cat0_purchase_date_max']).dt.days
df_test[f'{last_type}_cat1_term_from_now'] = (pd.to_datetime('2018-05-01') - df_test[f'{last_type}_cat1_purchase_date_max']).dt.days

In [8]:
# Null Impute
for col in df_train.columns:
    if col in ignore_list:continue
    if str(type(df_test[col].dtypes)).count('time'):
        df_train[col].fillna(df_train[col].min(), inplace=True)
        df_test[col].fillna(df_train[col].min(), inplace=True)

### Save Feature

In [9]:
fname = '166_fe3'
ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if col in ignore_features: continue
#     if (col.count('feature_')):continue
    if (col.count('purchase_date')):continue
    feat_train = df_train[col].astype('float32').values
    try:
        feat_test = df_test[col].astype('float32').values
    except TypeError:
        print(col)
        sys.exit()
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)

In [71]:
import glob
path_list = glob.glob('../features/1_first_valid/*.gz')
for path in path_list:
    length = utils.read_pkl_gzip(path).shape[0]
    if path.count('test'):
        print(length)
#         if length>220000:
#             print(path)
#     if path.count('test'):
#         if length>140000:
#             print(path)

SystemExit: 